In [1]:
import sys
import collections
from collections import Counter

import datetime
from datetime import datetime, timedelta, timezone
import dateutil.parser

from collections import defaultdict

import pandas as pd
import seaborn as sns
pd.options.display.max_rows = 999

import pydent
from pydent import AqSession, models
from pydent.models import Sample, Item, Plan

sys.path.append("../ext-plan-pydent")
from plans import ExternalPlan

In [2]:
filename = 'config.yml'

with open(filename, 'r') as f:
    config = yaml.load(f)

login = config['aquarium'][aq_instance]

session = AqSession(
    login['username'],
    login['password'],
    login['url']
)

me = session.User.where({'login': login['username']})[0]
print('Logged in as %s\n' % me.name)
    
session = ExternalPlan.create_session("production")

Logged in as Devin Strickland



### Get all Operations from a single month window

In [3]:
year = 2018
month = 11

month_start = datetime(year, month, 1, tzinfo=timezone(-timedelta(hours=7)))
month_end = datetime(year, month + 1, 1, tzinfo=timezone(-timedelta(hours=7)))

window_ops = session.Operation.where("updated_at >= '{}' AND updated_at < '{}'".format(month_start, month_end))
print("{} Operations found between {} and {}.".format(len(window_ops), month_start, month_end))

2018-11-01 00:00:00-07:00 Operations found between 2018-12-01 00:00:00-07:00 and 4529.


### Get PlanAssociations for those Operations

In [4]:
window_op_ids = [op.id for op in window_ops]
window_plan_associations = session.PlanAssociation.where({"operation_id": window_op_ids})
print("{} PlanAssociations found for {} Operations.".format(len(window_plan_associations), len(window_ops)))

4529 PlanAssociations found for 4529 Operations.


### Get Plans for those PlanAssociations that are also for the DARPA budget

In [5]:
budget_name = "DARPA"
darpa_budget = session.Budget.find_by_name(budget_name)
window_plan_ids = [pa.plan_id for pa in window_plan_associations]
window_darpa_plans = session.Plan.where({"budget_id": darpa_budget.id, "id": window_plan_ids})
print("{} Plans found for {} between {} and {}.".format(len(window_darpa_plans), budget_name, month_start, month_end))

125 Plans found for DARPA between 2018-11-01 00:00:00-07:00 and 2018-12-01 00:00:00-07:00.


### Get all Operations for only those Plans

In [6]:
window_darpa_plan_ids = [p.id for p in window_darpa_plans]
window_darpa_plan_associations = session.PlanAssociation.where({"plan_id": window_darpa_plan_ids})
window_darpa_op_ids = [pa.operation_id for pa in window_darpa_plan_associations]
window_darpa_ops = session.Operation.find(window_darpa_op_ids)
print("{} Operations found for {} Plans.".format(len(window_darpa_ops), budget_name))

3670 Operations found for DARPA Plans.


### Select only the Plans for which the last Operation was completed in the window

In [7]:
plan_data = {}

for plan in window_darpa_plans:
    pas = [pa for pa in window_darpa_plan_associations if pa.plan_id == plan.id]
    op_ids = [pa.operation_id for pa in pas]
    ops = [op for op in window_darpa_ops if op.id in op_ids]
    statuses = set([op.status for op in ops])
    
    # Exclude plans that are not fully errored or done.
    if statuses - {'done', 'error'}:
        continue
        
    # Exclude plans that are only errors.
    if not statuses - {'error'}:
        continue
    
    sorted_ops = sorted(ops, key=lambda x: x.updated_at)
    last_op_updated_at = dateutil.parser.parse(sorted_ops[-1].updated_at)
    
    if last_op_updated_at >= month_start and last_op_updated_at < month_end:
        d = {}
        d["plan"] = plan
        d["ops"] = sorted_ops
        d["error"] = 'error' in statuses
        plan_data[plan.id] = d
        
print("{} completed Plans found that end in the specified month.".format(len(plan_data)))
print("{} Plans have errored Operations.".format(len([p for p in plan_data.values() if p["error"]])))

83 completed Plans found that end in the specified month.
14 Plans have errored Operations.


### Get all OperationTypes and Users and put them in dicts for easy access

In [8]:
all_operation_types = {}
for ot in session.OperationType.all():
    all_operation_types[ot.id] = ot
    
all_users = {}
for user in session.User.all():
    all_users[user.id] = user

### Build a table

In [9]:
headers = [
    "Challenge problem",
    "Use case/Experiment",
    "Researcher",
    "Description",
    "Data volume delivered to TA4 this month (TBs)",
    "Cumulative data volume delivered to TA4 (TB)",
    "Method Development Required?",
    "Strains",
    "Protocol Performed",
    "Date Started",
    "Date Finished",
    "# Sample",
    "Measurement",
    "Data Volume generated (GBs)",
    "Date data delivered to TA4",
    "Data format",
    "Cost",
    "Comments"
]

def get_challenge_problem(ops, all_operation_types):
    op_types = list(set([all_operation_types[op.operation_type_id].name for op in ops]))
    if "Challenge and Label" in op_types:
        return "Protein Design"
    elif "Treat With Zymolyase" in op_types:
        return "Protein Design"
    elif "4. Measure OD and GFP" in op_types:
        return "YeastSTATES"
    
def get_data_stats(plan):
    

In [10]:
table_data = []

for id, data in plan_data.items():
    dates = [op.updated_at for op in data["ops"]]
    start = dateutil.parser.parse(min(dates))
    end = dateutil.parser.parse(max(dates))
    
    comments = []
    row = {}
    row["Description"] = data["plan"].name

    row["Date Started"] = start.strftime("%x")
    row["Date Finished"] = end.strftime("%x")

    if not data.get("cost"):
        data["cost"] = data["plan"].estimate_cost()

    row["Cost"] = data["cost"]
    
    row["Researcher"] = all_users[data["plan"].user_id].name

    if data["error"]:
        comments.append("Some errored Operations.")

    row["Challenge problem"] = get_challenge_problem(data["ops"], all_operation_types)
    
    get_data_stats = 

    row["Comments"] = " ".join(comments)
    table_data.append(row)

In [11]:
df = pd.DataFrame(data=table_data, columns=headers)

In [12]:
df

,Challenge problem,Use case/Experiment,Researcher,Description,Data volume delivered to TA4 this month (TBs),Cumulative data volume delivered to TA4 (TB),Method Development Required?,Strains,Protocol Performed,Date Started,Date Finished,# Sample,Measurement,Data Volume generated (GBs),Date data delivered to TA4,Data format,Cost,Comments
0,Protein Design,NaN,Devin Strickland,B1234 Trypsin Only,NaN,NaN,NaN,NaN,NaN,04/20/18,11/05/18,NaN,NaN,NaN,NaN,NaN,0.000000,
1,NaN,NaN,Eriberto Lopez,2Gene_pYMOD_2uVector_MAD7_Plasmid 091718,NaN,NaN,NaN,NaN,NaN,10/10/18,11/26/18,NaN,NaN,NaN,NaN,NaN,244.837484,Some errored ops.
2,YeastSTATES,NaN,Devin Strickland,Cell State Reporters Set 1 (replan),NaN,NaN,NaN,NaN,NaN,09/25/18,11/06/18,NaN,NaN,NaN,NaN,NaN,53.829560,Some errored ops.
3,NaN,NaN,Justin Vrana,gRNA Cassette Fragments 1/3,NaN,NaN,NaN,NaN,NaN,10/25/18,11/01/18,NaN,NaN,NaN,NaN,NaN,105.025553,
4,NaN,NaN,Justin Vrana,gRNA Cassette Fragments 2/3,NaN,NaN,NaN,NaN,NaN,10/25/18,11/01/18,NaN,NaN,NaN,NaN,NaN,105.025553,
5,NaN,NaN,Justin Vrana,gRNA Cassette Fragments 3/3,NaN,NaN,NaN,NaN,NaN,10/25/18,11/01/18,NaN,NaN,NaN,NaN,NaN,105.025553,
6,Protein Design,NaN,Devin Strickland,aga2_mutagenesis_1,NaN,NaN,NaN,NaN,NaN,10/26/18,11/05/18,NaN,NaN,NaN,NaN,NaN,0.000000,
7,Protein Design,NaN,Devin Strickland,tj_brunette_1_ngs_prep,NaN,NaN,NaN,NaN,NaN,10/29/18,11/05/18,NaN,NaN,NaN,NaN,NaN,81.434950,
8,NaN,NaN,Joe Control,Plasmid Assembly Control 10/24/2018 (copy) (co...,NaN,NaN,NaN,NaN,NaN,10/30/18,11/01/18,NaN,NaN,NaN,NaN,NaN,12.541114,
9,NaN,NaN,Samer Halabiya,4 gDNA ngs prep for promoter lib,NaN,NaN,NaN,NaN,NaN,10/31/18,11/05/18,NaN,NaN,NaN,NaN,NaN,0.000000,Some errored ops.


In [ ]:
df.to_csv("progress_report.csv", index=False)

In [ ]:
print(window_darpa_plans[-1])